# Parallel Sub-Agent Research

This notebook demonstrates **parallel sub-agent delegation** - spawning multiple agents to research sub-questions simultaneously.

| Pattern | Notebook 01 | This Notebook |
|---------|:-----------:|:-------------:|
| Sequential searches | ✅ | ❌ |
| Parallel sub-agents | ❌ | ✅ |
| Isolated context per question | ❌ | ✅ |
| Speed | Slower | **Faster** |

**Architecture:**
```
                    ┌─────────────┐
                    │   Planner   │
                    │   Agent     │
                    └──────┬──────┘
                           │ Creates sub-questions
           ┌───────────────┼───────────────┐
           ▼               ▼               ▼
    ┌────────────┐  ┌────────────┐  ┌────────────┐
    │ Sub-Agent  │  │ Sub-Agent  │  │ Sub-Agent  │
    │     #1     │  │     #2     │  │     #3     │
    │ (Question) │  │ (Question) │  │ (Question) │
    └─────┬──────┘  └─────┬──────┘  └─────┬──────┘
          │               │               │
          └───────────────┼───────────────┘
                          ▼
                   ┌─────────────┐
                   │ Synthesizer │
                   │   Agent     │
                   └─────────────┘
```

## Step 1: Setup

In [ ]:
#!pip install openhands-sdk litellm python-dotenv tavily-python

In [ ]:
import os
import json
import asyncio
from concurrent.futures import ThreadPoolExecutor
from dotenv import load_dotenv
load_dotenv()

from typing import List
from pydantic import Field
from tavily import TavilyClient
from openhands.sdk import LLM, Agent, Conversation, Tool, Action, Observation, ToolDefinition, TextContent
from openhands.sdk.tool import register_tool, ToolExecutor
from openhands.tools.file_editor import FileEditorTool

print(f"Model: {os.getenv('LLM_MODEL', 'openai/gpt-4o')}")

# Observability status
if os.getenv("LMNR_PROJECT_API_KEY"):
    print("✓ Observability: Laminar tracing enabled")
else:
    print("ℹ Observability: Set LMNR_PROJECT_API_KEY for tracing")

## Step 2: Create LLM and Tavily Tool

In [ ]:
# Create LLMs
llm = LLM(
    model="openai/gpt-4o",
    api_key=os.getenv("LLM_API_KEY"),
    base_url=os.getenv("LLM_BASE_URL", None),
)

synthesis_llm = LLM(
    model="openai/gpt-5.1",
    api_key=os.getenv("LLM_API_KEY"),
    base_url=os.getenv("LLM_BASE_URL", None),
)

# Tavily client and tool
tavily = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))

class SearchAction(Action):
    query: str = Field(description="Search query")

class SearchObservation(Observation):
    results: str = Field(description="Results")
    @property
    def to_llm_content(self): return [TextContent(text=self.results)]

class SearchExecutor(ToolExecutor):
    def __call__(self, action: SearchAction, conversation=None) -> SearchObservation:
        try:
            r = tavily.search(query=action.query, max_results=5)
            text = "\n\n".join([
                f"**{x['title']}**\n{x['content'][:300]}\nSource: {x['url']}"
                for x in r['results']
            ])
            return SearchObservation(results=text or "No results found")
        except Exception as e:
            return SearchObservation(results=f"Search failed: {str(e)}")

class SearchTool(ToolDefinition[SearchAction, SearchObservation]):
    @classmethod
    def create(cls, conv_state) -> List["SearchTool"]:
        return [cls(description="Web search via Tavily",
                    action_type=SearchAction, observation_type=SearchObservation, 
                    executor=SearchExecutor())]

register_tool("TavilySearch", SearchTool.create)
print("✓ LLMs and Tavily tool ready")

## Step 3: Define Sub-Agent Research Function

Each sub-agent researches ONE sub-question independently.

In [ ]:
cwd = os.getcwd()

def research_sub_question(question: str, index: int) -> dict:
    """
    Spawn a sub-agent to research a single sub-question.
    Each agent writes its findings to a unique file.
    """
    print(f"  🔍 Sub-agent {index+1} starting: {question[:50]}...")
    
    # Create a dedicated agent for this sub-question
    sub_agent = Agent(
        llm=llm,
        tools=[Tool(name="TavilySearch"), Tool(name=FileEditorTool.name)],
    )
    
    # Create isolated conversation for this sub-agent
    sub_conversation = Conversation(agent=sub_agent, workspace=cwd)
    
    # Each sub-agent writes to its own file
    output_file = f"findings_{index+1}.md"
    
    prompt = f"""
    Research this specific question: {question}
    
    1. Use TavilySearch to find 2-3 relevant sources
    2. Write your findings to the file `{output_file}` including:
       - Key points discovered
       - Source URLs for citations
    
    Be concise but thorough.
    """
    
    sub_conversation.send_message(prompt)
    sub_conversation.run()
    
    # Read findings from the file the agent created
    findings = ""
    try:
        with open(output_file, "r") as f:
            findings = f.read()
    except FileNotFoundError:
        findings = f"[Sub-agent {index+1} did not write findings]"
    
    print(f"  ✓ Sub-agent {index+1} complete")
    
    return {
        "question": question,
        "findings": findings,
        "index": index
    }

print("✓ Sub-agent research function ready")

## Step 4: Run Parallel Research

Uses ThreadPoolExecutor to run sub-agents in parallel.

In [ ]:
import time

# Research topic and sub-questions
topic = "Latest breakthroughs in AI agents and autonomous systems (2024-2025)"

sub_questions = [
    "What are the major advances in LLM-based autonomous agents in 2024-2025?",
    "How has multi-agent collaboration evolved in AI systems?",
    "What new tools and frameworks have emerged for building AI agents?",
    "What are the key challenges and limitations of current AI agents?",
    "What real-world applications are using AI agents successfully?",
]

print("=" * 60)
print(f"PARALLEL RESEARCH: {topic}")
print("=" * 60)
print(f"\nResearching {len(sub_questions)} sub-questions in parallel...\n")

# Run sub-agents in parallel using ThreadPoolExecutor
start_time = time.time()

with ThreadPoolExecutor(max_workers=len(sub_questions)) as executor:
    # Submit all sub-agent tasks
    futures = [
        executor.submit(research_sub_question, q, i) 
        for i, q in enumerate(sub_questions)
    ]
    
    # Collect results as they complete
    results = [f.result() for f in futures]

elapsed = time.time() - start_time
print(f"\n✓ All {len(sub_questions)} sub-agents complete in {elapsed:.1f}s")

# Sort by original index
results.sort(key=lambda x: x['index'])

# Save raw findings to file
findings_text = f"# Research Findings: {topic}\n\n"
for r in results:
    findings_text += f"## {r['index']+1}. {r['question']}\n\n"
    findings_text += f"{r['findings']}\n\n---\n\n"

with open("parallel_findings.md", "w") as f:
    f.write(findings_text)

print("✓ Raw findings saved to parallel_findings.md")

## Step 5: Synthesize with GPT-5.1

In [ ]:
print("=" * 60)
print("SYNTHESIS: Creating comprehensive report (GPT-5.1)...")
print("=" * 60)

# Create synthesis agent
synthesis_agent = Agent(
    llm=synthesis_llm,
    tools=[Tool(name=FileEditorTool.name)],
)

synthesis_conversation = Conversation(agent=synthesis_agent, workspace=cwd)

SYNTHESIS_PROMPT = """
Read `parallel_findings.md` which contains research findings from multiple parallel searches.

Synthesize these into a comprehensive report and write it to `parallel_report.md`:

# [Topic Title]

## Executive Summary
(2-3 paragraph overview)

## Key Findings
(Organize by theme, not by original question order)
(Cite sources inline)

## Analysis & Implications
(What do these findings mean?)

## Conclusion

## References
(All sources with URLs)

Write professionally. Synthesize and connect ideas across the different sub-questions.
"""

synthesis_conversation.send_message(SYNTHESIS_PROMPT)
synthesis_conversation.run()

print("\n✓ Synthesis complete!")

## Step 6: View the Report

In [ ]:
from IPython.display import Markdown, display

try:
    with open("parallel_report.md", "r") as f:
        display(Markdown(f.read()))
except FileNotFoundError:
    print("parallel_report.md not found - run the cells above first")

## Summary: Sequential vs Parallel

| Approach | Notebook 01 | This Notebook |
|----------|-------------|---------------|
| **Execution** | Sequential | Parallel (ThreadPoolExecutor) |
| **Sub-agents** | 1 agent, multiple turns | N agents, isolated context |
| **Speed** | ~N × search_time | ~1 × search_time |
| **Context** | Shared (can grow large) | Isolated (stays small) |
| **Error handling** | One failure stops all | Failures isolated |

**When to use parallel:**
- Many independent sub-questions
- Speed is important
- Sub-questions don't depend on each other

**When to use sequential (Notebook 01):**
- Sub-questions build on each other
- Need to refine queries based on earlier results
- Simpler debugging